# Imports

In [25]:
import os
import wandb
import torch

# Download and Save Raw Data

In [3]:
# Download the training data
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
!mkdir -p ../data/
!mv input.txt ../data/input.txt

--2023-12-15 23:25:27--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.09s   

2023-12-15 23:25:27 (12.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [9]:
# Load environment variables
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [10]:
# Upload the data as an artifact to weights and biases
run = wandb.init(name='upload-mini-shakespeare', job_type='upload')
artifact = wandb.Artifact('mini-shakespeare-text', type='raw_data')
artifact.add_file(
    '../data/input.txt',
    name='input.txt'
)
wandb.log_artifact(artifact)
run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: martmichals (martymcfly). Use `wandb login --relogin` to force relogin


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [12]:
# Load the data
with open('../data/input.txt', 'r', encoding='utf-8') as inf:
    text = inf.read()
print(f'Length of dataset in characters: {len(text)}')

Length of dataset in characters: 1115394


In [14]:
# Show the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [18]:
# Set up the vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f'Vocabulary size: {len(chars)}')


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocabulary size: 65


In [22]:
# Set up the encoder and decoder
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[ch] for ch in s] # str -> List[int]
decode = lambda i: ''.join([itos[i] for i in l])   # List[int] -> str

In [23]:
# Show the encoder and decoder in action
print(l:=encode('Hello there!'))
print(decode(l))

[20, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43, 2]
Hello there!


In [26]:
# Encode the dataset into a sequence of integers
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [28]:
# Split the data into a train and validation set
n = int(0.9*len(data))
train = data[:n]
val = data[n:]

In [29]:
# Save the tensors
torch.save(train, '../data/train.pt')
torch.save(val, '../data/val.pt')

# Upload the data as an artifact to weights and biases
run = wandb.init(name='upload-mini-shakespeare-train-test', job_type='upload')
original_artifact = run.use_artifact('nanogpt/mini-shakespeare-text:latest')
artifact = wandb.Artifact('mini-shakespeare-tensors', type='raw_data')
artifact.add_file(
    '../data/train.pt',
    name='train.pt'
)
artifact.add_file(
    '../data/val.pt',
    name='val.pt'
)
wandb.log_artifact(artifact)
run.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
